In [4]:
import torch
from src.utils.model import freeze_layer
state_dict = torch.load('weight\model_finetune_arcface_k=1_best.pth', map_location= 'cpu')['model_state_dict']

In [15]:
model = torch.load(r'weight\backbone_r100_glint360k.pt',  map_location= 'cpu')

freeze_layer(model, ['253', '254'])

In [ ]:
model.train()

In [ ]:
for name, v in model.named_parameters():
    print(name, v.requires_grad)

In [ ]:
for name, v in model.named_modules():
    print(name, v.training)

In [ ]:
for k in state_dict.keys():
    print(k)